In [1]:
import os
os.chdir('..') # this resolves ImportError: attempted relative import with no known parent package
# import sys
# parent_dir = os.getcwd()
# sys.path.append(parent_dir)

In [2]:
import pandas as pd

In [3]:
raw_data = pd.read_csv("data/processed/json_data_processed.csv")
data = raw_data.copy()
df = pd.DataFrame(data)
df.head()

,videoID,publishedAt,channelId,title,description,channelTitle,tags,viewCount,likeCount,favoriteCount,commentCount,duration_timedelta,duration_hhmmss
0,uGtc9Bu9Txk,2020-12-31T23:00:10Z,UCArk93C2pbOvkv6jWz-3kAg,🍾 FORD KIERNAN HOGMANAY SPECIAL! | Open Goal,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,Open Goal,"['open goal', 'si ferry', 'still game', 'jack ...",205682,2286,0,112,0 days 01:21:55,01:21:55
1,vAGmV-mRRT0,2020-12-24T07:00:08Z,UCArk93C2pbOvkv6jWz-3kAg,CHRISTMAS EVE SPECIAL | Right in the Coupon w/...,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,Open Goal,"['open goal', 'si ferry', 'betting', 'slaney',...",63357,742,0,30,0 days 00:44:31,00:44:31
2,m54LPZSlStQ,2020-12-22T17:00:20Z,UCArk93C2pbOvkv6jWz-3kAg,EPIC CUP FINAL REVIEW | Keeping the Ball on th...,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,Open Goal,"['open goal', 'si ferry', 'celtic vs hearts', ...",85262,976,0,63,0 days 01:06:12,01:06:12
3,1RcFj1Sfs58,2020-12-21T17:00:03Z,UCArk93C2pbOvkv6jWz-3kAg,JAMIE CARRAGHER | Open Goal Meets...,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,Open Goal,"['open goal', 'si ferry', 'Jamie Carragher', '...",855101,4966,0,661,0 days 02:22:14,02:22:14
4,MaJhyEr6csQ,2020-12-17T17:25:28Z,UCArk93C2pbOvkv6jWz-3kAg,CELTIC v HEARTS & ST MIRREN END RANGERS UNBEAT...,SUBSCRIBE to Open Goal - https://bit.ly/2QGY26...,Open Goal,"['open goal', 'si ferry', 'celtic v hearts', '...",64383,659,0,49,0 days 00:45:40,00:45:40


In [4]:
# define a function that counts the number of videos released in a calendar month
def create_year_month_columns(df):
    """Calculates the number of videos per month and per year based on the date in the 'publishedAt' column.
    
    Parameters
    -----------
    df : pandas dataframe
        Dataframe with 'publishedAt' column.
        
    Returns
    -------
    df : pandas dataframe
        The input dataframe with additional 'publishedAtYear', 'publishedAtMonth', and 'publishedAtYearMonth columns.
        
    Notes
    ------
    
    
    """
    # create copy of input dataframe to avoid any inplace changes
    df_ = df.copy()
    
    # convert publishedAt values from string to datetime
    df_['publishedAt'] = pd.to_datetime(df_['publishedAt'])
    
    # use strftime to format the datetime value into just year and month so we can group by year and month
    df_['publishedAtYear'] = df_['publishedAt'].dt.strftime("%Y")
    df_['publishedAtMonth'] = df_['publishedAt'].dt.strftime("%m")
    df_['publishedAtYearMonth'] = df_['publishedAt'].dt.strftime("%Y-%m")

    # sort values and reset index
    df_.sort_values(by=['publishedAt'], inplace=True)
    df_.reset_index(drop=True, inplace=True)

    try:
        df_ = df_[['videoID', 'publishedAt', 'publishedAtYear', 'publishedAtMonth', 'publishedAtYearMonth', 
               'channelTitle', 'channelId',  'title', 'description', 'duration_timedelta', 'duration_hhmmss', 'tags', 
               'viewCount', 'likeCount', 'favoriteCount','commentCount']]

    except KeyError as ke:
        # store error message (format is "KeyError: ['column_x', 'column_y', ... 'column_n'] not in index")
        error_message = str(ke)     
        
        # Extract the string between square brackets
        missing_columns_str = error_message.split("'")[1]
        
        # Split the string between the square brackets into individual column names
        missing_columns_list = missing_columns_str.split(", ")
        
        # Print the list of missing columns
        print(f"Warning: {ke}\nDataframe created without these columns.")

        # store remaining columns
        remaining_columns = [col for col in df_.columns if col not in missing_columns_list]
        
        # create dataframe with just remaining columns
        df_ = df_[remaining_columns]


    return df_

# Test function with all columns present

In [5]:
df = create_year_month_columns(df)
df.head(2)

,videoID,publishedAt,publishedAtYear,publishedAtMonth,publishedAtYearMonth,channelTitle,channelId,title,description,duration_timedelta,duration_hhmmss,tags,viewCount,likeCount,favoriteCount,commentCount
0,wA7v2RtVdSA,2017-05-26 12:49:13+00:00,2017,05,2017-05,Open Goal,UCArk93C2pbOvkv6jWz-3kAg,Open Goal Trailer,NaN,0 days 00:00:48,00:00:48,"['open goal', 'kevin thomson', ""darren o'dea"",...",8709,37,0,2
1,Fkmm0E_qgm8,2017-06-02 08:49:46+00:00,2017,06,2017-06,Open Goal,UCArk93C2pbOvkv6jWz-3kAg,Si Ferry Meets...Kevin Thomson Episode 1 - Exc...,"In Part 1 of Si Ferry Meets… Kevin Thomson, th...",0 days 00:06:45,00:06:45,"['Kevin Thomson', 'Rangers', 'Rangers Fc', 'Hi...",47975,152,0,7


In [6]:
df.columns

Index(['videoID', 'publishedAt', 'publishedAtYear', 'publishedAtMonth',
       'publishedAtYearMonth', 'channelTitle', 'channelId', 'title',
       'description', 'duration_timedelta', 'duration_hhmmss', 'tags',
       'viewCount', 'likeCount', 'favoriteCount', 'commentCount'],
      dtype='object')

# Test try-except block with missing columns

In [7]:
# test function if one column doesn't exist
df.drop(labels=["duration_timedelta"], axis=1, inplace=True)
df.head(2)

,videoID,publishedAt,publishedAtYear,publishedAtMonth,publishedAtYearMonth,channelTitle,channelId,title,description,duration_hhmmss,tags,viewCount,likeCount,favoriteCount,commentCount
0,wA7v2RtVdSA,2017-05-26 12:49:13+00:00,2017,05,2017-05,Open Goal,UCArk93C2pbOvkv6jWz-3kAg,Open Goal Trailer,NaN,00:00:48,"['open goal', 'kevin thomson', ""darren o'dea"",...",8709,37,0,2
1,Fkmm0E_qgm8,2017-06-02 08:49:46+00:00,2017,06,2017-06,Open Goal,UCArk93C2pbOvkv6jWz-3kAg,Si Ferry Meets...Kevin Thomson Episode 1 - Exc...,"In Part 1 of Si Ferry Meets… Kevin Thomson, th...",00:06:45,"['Kevin Thomson', 'Rangers', 'Rangers Fc', 'Hi...",47975,152,0,7


In [8]:
# try the function with missing column to see if try-except block works
df = create_year_month_columns(df)
df.head()

Dataframe created without these columns.


,videoID,publishedAt,publishedAtYear,publishedAtMonth,publishedAtYearMonth,channelTitle,channelId,title,description,duration_hhmmss,tags,viewCount,likeCount,favoriteCount,commentCount
0,wA7v2RtVdSA,2017-05-26 12:49:13+00:00,2017,05,2017-05,Open Goal,UCArk93C2pbOvkv6jWz-3kAg,Open Goal Trailer,NaN,00:00:48,"['open goal', 'kevin thomson', ""darren o'dea"",...",8709,37,0,2
1,Fkmm0E_qgm8,2017-06-02 08:49:46+00:00,2017,06,2017-06,Open Goal,UCArk93C2pbOvkv6jWz-3kAg,Si Ferry Meets...Kevin Thomson Episode 1 - Exc...,"In Part 1 of Si Ferry Meets… Kevin Thomson, th...",00:06:45,"['Kevin Thomson', 'Rangers', 'Rangers Fc', 'Hi...",47975,152,0,7
2,Ks0ZO6SGlsU,2017-06-02 08:50:46+00:00,2017,06,2017-06,Open Goal,UCArk93C2pbOvkv6jWz-3kAg,Si Ferry Meets...Kevin Thomson Episode 2 - The...,Part 2 – Kevin Thomson speaks with Si about th...,00:07:23,"['open goal', 'si ferry', 'si ferry kevin thom...",36476,115,0,11
3,-du5xqL3YcA,2017-06-02 11:40:56+00:00,2017,06,2017-06,Open Goal,UCArk93C2pbOvkv6jWz-3kAg,Si Ferry Meets...Charlie Mulgrew Episode 1 - C...,"In Part 1 of Si Ferry Meets… Charlie Mulgrew, ...",00:07:49,"['Charlie Mulgrew', 'Celtic', 'Celtic fc', 'ab...",69209,240,0,7
4,mfOy5Of3Tsg,2017-06-02 11:41:08+00:00,2017,06,2017-06,Open Goal,UCArk93C2pbOvkv6jWz-3kAg,Si Ferry Meets...Charlie Mulgrew Episode 2 - A...,Part 2 – Charlie tells Si about his positive t...,00:08:32,"['Charlie Mulgrew', 'Celtic', 'Celtic FC', 'Ab...",54063,162,0,7


In [9]:
df.columns

Index(['videoID', 'publishedAt', 'publishedAtYear', 'publishedAtMonth',
       'publishedAtYearMonth', 'channelTitle', 'channelId', 'title',
       'description', 'duration_hhmmss', 'tags', 'viewCount', 'likeCount',
       'favoriteCount', 'commentCount'],
      dtype='object')

In [10]:
# try the function with multiple missing columns to see if try-except block works
df.drop(labels=["duration_hhmmss",'tags', 'favoriteCount'], axis=1, inplace=True)

In [11]:
df.columns

Index(['videoID', 'publishedAt', 'publishedAtYear', 'publishedAtMonth',
       'publishedAtYearMonth', 'channelTitle', 'channelId', 'title',
       'description', 'viewCount', 'likeCount', 'commentCount'],
      dtype='object')

In [12]:
df = create_year_month_columns(df)
df.head()

Dataframe created without these columns.


,videoID,publishedAt,publishedAtYear,publishedAtMonth,publishedAtYearMonth,channelTitle,channelId,title,description,viewCount,likeCount,commentCount
0,wA7v2RtVdSA,2017-05-26 12:49:13+00:00,2017,05,2017-05,Open Goal,UCArk93C2pbOvkv6jWz-3kAg,Open Goal Trailer,NaN,8709,37,2
1,Fkmm0E_qgm8,2017-06-02 08:49:46+00:00,2017,06,2017-06,Open Goal,UCArk93C2pbOvkv6jWz-3kAg,Si Ferry Meets...Kevin Thomson Episode 1 - Exc...,"In Part 1 of Si Ferry Meets… Kevin Thomson, th...",47975,152,7
2,Ks0ZO6SGlsU,2017-06-02 08:50:46+00:00,2017,06,2017-06,Open Goal,UCArk93C2pbOvkv6jWz-3kAg,Si Ferry Meets...Kevin Thomson Episode 2 - The...,Part 2 – Kevin Thomson speaks with Si about th...,36476,115,11
3,-du5xqL3YcA,2017-06-02 11:40:56+00:00,2017,06,2017-06,Open Goal,UCArk93C2pbOvkv6jWz-3kAg,Si Ferry Meets...Charlie Mulgrew Episode 1 - C...,"In Part 1 of Si Ferry Meets… Charlie Mulgrew, ...",69209,240,7
4,mfOy5Of3Tsg,2017-06-02 11:41:08+00:00,2017,06,2017-06,Open Goal,UCArk93C2pbOvkv6jWz-3kAg,Si Ferry Meets...Charlie Mulgrew Episode 2 - A...,Part 2 – Charlie tells Si about his positive t...,54063,162,7
